In [268]:
import pandas as pd
import numpy as np

import datetime as dt

from scipy.stats import ttest_ind, ttest_rel, ttest_1samp
from scipy.stats import f_oneway, pearsonr, chisquare, chi2_contingency
from scipy.stats import shapiro, levene, bartlett

In [269]:
# 데이터 불러오기
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_1.csv')

df.head()

,구급보고서번호,소방서명,환자증상구분명1,신고일자,신고시각,출동일자,출동시각,환자직업명,사고발생장소
0,20211122103M01526,중랑소방서,심계항진,20210330,104800,20210330,105042,무직,집
1,20211111105M00771,노원소방서,복통,20210412,225600,20210412,225827,기타,집
2,20211115104M00864,서초소방서,심계항진,20210303,194100,20210303,194500,직장인,상업시설
3,20211113107M01869,마포소방서,고열,20211225,235400,20211226,331,유아,집
4,20211118102M05820,강동소방서,고열,20211019,12100,20211019,12322,무직,집


In [270]:
df['신고일자'] = df['신고일자'].astype('str')
df['신고시각'] = df['신고시각'].astype('str')

df['신고시각'] = df['신고시각'].map(lambda x:str(x).zfill(6))

df['신고_DATE'] = df['신고일자'] + df['신고시각']

print(df['신고_DATE'].head(10))

0    20210330104800
1    20210412225600
2    20210303194100
3    20211225235400
4    20211019012100
5    20210810122800
6    20211024091500
7    20210315235900
8    20210228091500
9    20210729124900
Name: 신고_DATE, dtype: object


In [271]:
df['출동일자'] = df['출동일자'].astype('str')
df['출동시각'] = df['출동시각'].astype('str')

df['출동시각'] = df['출동시각'].map(lambda x:str(x).zfill(6))

df['출동_DATE'] = df['출동일자'] + df['출동시각']

print(df['출동_DATE'].head(10))

0    20210330105042
1    20210412225827
2    20210303194500
3    20211226000331
4    20211019012322
5    20210810122900
6    20211024091725
7    20210316000239
8    20210228091900
9    20210729125146
Name: 출동_DATE, dtype: object


In [272]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2762 entries, 0 to 2761
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   구급보고서번호   2762 non-null   object
 1   소방서명      2762 non-null   object
 2   환자증상구분명1  2762 non-null   object
 3   신고일자      2762 non-null   object
 4   신고시각      2762 non-null   object
 5   출동일자      2762 non-null   object
 6   출동시각      2762 non-null   object
 7   환자직업명     2762 non-null   object
 8   사고발생장소    2762 non-null   object
 9   신고_DATE   2762 non-null   object
 10  출동_DATE   2762 non-null   object
dtypes: object(11)
memory usage: 237.5+ KB


##### 문제 1-1
- 각 구급보고서별 출동시각과 신고시각의 차이 -> '소요시간' 컬럼으로 추가 (초 단위)
- 소방서별 소요시간의 평균을 오름차순으로 정렬했을 때
- 3번째로 작은 소요시간 값과 소방서명을 출력하라

In [273]:
df['신고_DATE'] = pd.to_datetime(df['신고_DATE'])
df['출동_DATE'] = pd.to_datetime(df['출동_DATE'])

df['소요시각'] = df['출동_DATE']-df['신고_DATE']

In [274]:
# df['소요시각'] = (
#                 pd.to_datetime(
#                     df['출동일자'].astype('str') + df['출동시각'].astype('str').str.zfill(6)
#                 )

#                 - 
    
#                 pd.to_datetime(
#                     df['신고일자'].astype('str') + df['신고시각'].astype('str').str.zfill(6)
#                 )
#                 ).dt.total_seconds()


result = df.groupby(['소방서명'])['소요시각'].mean().sort_values().reset_index().iloc[2].values
print(result)

['종로소방서' Timedelta('0 days 00:02:55.500000')]
